In [1]:
%matplotlib widget
import os
import numpy as np
from scipy import linalg, spatial, fft, signal
from sklearn.experimental import enable_iterative_imputer
from sklearn import metrics, decomposition, impute
import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon, Rectangle
from PIL import Image, ExifTags, ImageFile
import visualize as vis
import CR_search as cr
import preprocessing as pp
import calculate as cc
from read_WDF_class import WDF

ImageFile.LOAD_TRUNCATED_IMAGES = True

folder = os.path.expanduser("~/Documents/RamanData/")
subfolder = "Dejan/testing_reader/"
subfolder = "Chloe/prob/"
files = os.listdir(os.path.join(folder,subfolder))

for i, tf in enumerate(files):
    if tf[-3:] == 'wdf':
        print(i, ') ', tf)  

0 )  LFeige-x20-532streamline-carto2.wdf
1 )  LFeige-532streamline-x20-speedmode-carto1.wdf
2 )  sableindien-x5-532streamline-map1.wdf


In [2]:
file = files[1]
filename = os.path.join(folder,subfolder,file)

fff = WDF(filename, verbose=True)

Reading the file: "LFeige-532streamline-x20-speedmode-carto1.wdf"


================================= Block : WDF1 =================================
size: 512, offset: 0
WdfFlag--------------------------------- : 	UnknownFlag (check in WiRE?)
PointsPerSpectrum----------------------- : 	1015
Capacity-------------------------------- : 	599000
Count----------------------------------- : 	599000
AccumulationCount----------------------- : 	1
YlistLength----------------------------- : 	1
XlistLength----------------------------- : 	1015
DataOriginCount------------------------- : 	6
ApplicationName------------------------- : 	WiRE
ApplicationVersion---------------------- : 	5.3.0 build 16195
ScanType-------------------------------- : 	StreamLine
MeasurementType------------------------- : 	Map
StartTime------------------------------- : 	Wed Mar 10 14:54:12 2021
EndTime--------------------------------- : 	Wed Mar 10 21:34:48 2021
SpectralUnits--------------------------- : 	Counts
LaserWaveLength-

In [31]:
np.size(fff.img_arr, 0) / float(fff.img_exif["FocalPlaneYResolution"]),\
np.size(fff.img_arr, 1) / float(fff.img_exif["FocalPlaneXResolution"])

(0.17119993844496595, 0.17116055203575464)

In [39]:
(fff.ysizepx-1)/(3.1*0.17119993844496595)

599.1862268938021

In [17]:
fff.img_exif

{'FocalPlaneXYOrigins': (-9818.9, -5364.7),
 'FieldOfViewXY': (4072.2, 2599.3),
 'px/µ ?': 1.0,
 'ExifOffset': 114,
 'ImageDescription': 'white-light image',
 'Make': 'Renishaw',
 'FocalPlaneResolutionUnit': 'Micron',
 'FocalPlaneXResolution': 4072.2,
 'FocalPlaneYResolution': 2599.3}

In [33]:
fff.show_grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fff.map_params

{'MapAreaType': 'StreamLine',
 'InitialCoordinates': array([-9659.71, -3301.61,   -16.  ], dtype=float32),
 'StepSizes': array([ 3.1, -3.1,  1. ], dtype=float32),
 'NbSteps': array([1000,  599,    1], dtype=uint32),
 'LineFocusSize': 0}

In [12]:
fff.img_exif

{'FocalPlaneXYOrigins': (-9818.9, -5364.7),
 'FieldOfViewXY': (4072.2, 2599.3),
 'px/µ ?': 1.0,
 'ExifOffset': 114,
 'ImageDescription': 'white-light image',
 'Make': 'Renishaw',
 'FocalPlaneResolutionUnit': 'Micron',
 'FocalPlaneXResolution': 4072.2,
 'FocalPlaneYResolution': 2599.3}

In [34]:
18.7*215/4.448843930635838

903.7179237315663

In [33]:
3078.6/692, 1968.9/443

(4.448843930635838, 4.444469525959368)

In [6]:
fff.xsizepx, fff.xminpx, fff.xmaxpx, np.size(fff.img_arr, 1)

(532, 27, 558, 697)

In [4]:
plt.figure()
plt.imshow(fff.img_arr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fff.img_reduced.shape

(104, 38, 3)

In [9]:
fff.n_y, fff.n_x

(599, 1000)

In [10]:
(fff.yminpx, fff.ymaxpx), (fff.xminpx, fff.xmaxpx), fff.img_arr.shape

((35, 353), (27, 558), (445, 697, 3))

In [8]:
fff.img_reduced.shape

(106, 208, 3)

In [4]:
plt.close('all')
fig, (ax1, ax_org) = plt.subplots(ncols=2)
ax1.imshow(fff.img_reduced)
ax_org.imshow(fff.img_arr[fff.yminpx:fff.ymaxpx+1,fff.xminpx:fff.xmaxpx+1,:])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
trta = vis.ShowSelected(np.hstack((fff.img_reduced.reshape(-1, 3)/np.median(fff.img_reduced)*np.median(fff.spectra[:,:10]), fff.spectra)).reshape(fff.n_y, fff.n_x, -1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
np.median(fff.spectra, axis=1)

array([10212.554 , 10576.049 , 10508.971 , 10517.353 , 10088.966 ,
        9713.409 , 10719.47  , 10113.2295, 14662.052 , 11676.192 ,
       11473.28  ,  9795.42  , 10505.303 , 10401.177 , 10399.374 ,
        9967.6045, 10464.077 , 11181.056 , 11032.9795, 11681.709 ,
       11244.727 , 11423.2705, 10196.805 , 10624.375 , 10633.829 ,
       10370.874 ,  9613.456 ,  9391.6455,  9688.78  , 10371.386 ,
       11098.79  , 10713.802 , 11102.837 , 10183.155 , 10550.289 ,
       10638.072 , 10362.739 ,  9629.221 , 11548.878 , 13020.814 ,
       18916.959 , 26236.5   , 16105.068 , 14970.9   , 10166.18  ,
       10578.855 , 10578.056 , 10056.657 , 10316.881 , 21705.17  ,
       13661.37  , 14454.053 , 16155.013 , 20144.463 , 22481.896 ,
       10125.651 , 10590.541 , 10577.556 ,  9706.841 , 17678.283 ,
       14843.166 ,  9907.976 , 10571.16  , 10950.125 , 11577.157 ,
       11868.393 , 10105.922 , 10524.687 , 10376.984 ,  9368.752 ,
       18678.293 ,  8866.898 , 10076.01  , 10416.8125, 10825.1

In [93]:
330/56, 101/17, 3.1/xres, 3.1/yres

(5.892857142857143, 5.9411764705882355, 5.898785425101214, 5.887240356083086)

In [87]:

nd.median_filter(slika, size=)
fff.img_arr[yminpx:ymaxpx+1, xminpx:xmaxpx+1,:].shape

TypeError: mean() missing 1 required positional argument: 'input'

In [80]:
trta.aximg.imshow(fff.img_arr[yminpx:ymaxpx, xminpx:xmaxpx,:], alpha=0.1)

In [90]:
7904/1580.8, 5055/1011

(5.0, 5.0)

In [46]:
blocks = list(zip(fff.block_names, fff.b_off, fff.block_sizes))
blocks.sort(key=lambda y: y[1])
for n,o,s in blocks:
    print(f"{n} : {o:>12d} -> {s:>12d}")

WDF1 :            0 ->          512
DATA :          512 ->       660676
YLST :       661188 ->           28
XLST :       661216 ->         4028
WXDA :       665244 ->         2716
ORGN :       667960 ->         6740
TEXT :       674700 ->           78
WXDM :       674778 ->       313495
WHTL :       988273 ->        82652
WMAP :      1070925 ->           64
BKXL :      1070989 ->         4044
WXCS :      1075033 ->        10855
WXIS :      1085888 ->         3915
ZLDC :      1089803 ->          314
WXDB :      1090117 ->     26422126
WMSK :     27512243 ->          290


In [47]:
fff.origins

,X (Micron),Y (Micron),Time (FileTime),Flags (Arbitrary),Checksum (Arbitrary)
0,-13.24,-30.0,1.327445e+17,0.0,0.0
1,-9.24,-30.0,1.327445e+17,0.0,0.0
2,-5.24,-30.0,1.327445e+17,0.0,0.0
3,-1.24,-30.0,1.327445e+17,0.0,0.0
4,2.76,-30.0,1.327445e+17,0.0,0.0
...,...,...,...,...,...
160,10.76,26.0,1.327445e+17,0.0,0.0
161,14.76,26.0,1.327445e+17,0.0,0.0
162,18.76,26.0,1.327445e+17,0.0,0.0
163,22.76,26.0,1.327445e+17,0.0,0.0


In [13]:
fff.origins["X (Micron)"].to_numpy().shape

(46225,)

In [17]:
plt.close("all")
plt.figure()
plt.plot(fff.origins["Y (Micron)"].to_numpy()[:1000])#.reshape(fff.n_x, -1)[0, :], '-o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fff.origins.columns

Index(['X (Micron)', 'Y (Micron)', 'Z (Micron)', 'Time (FileTime)',
       'Flags (Arbitrary)', 'Checksum (Arbitrary)'],
      dtype='object')

In [21]:
plt.figure()
plt.imshow(fff.origins["Z (Micron)"].to_numpy().reshape(fff.n_y, fff.n_x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
[x/6.32 for x in img_arr.shape]

[140.98101265822785, 141.13924050632912, 0.47468354430379744]

In [123]:
dd = np.arange(1,5)
all_phi = [xx for d in dd for xx in np.arange(np.pi/(3*d), 2*np.pi, np.pi/(3*d))]

for xx in all_phi:
    print(np.round(np.round((3*xx/np.pi), 2)%1, 2)>0)

n = 2*np.sqrt(2)
centri = [(2*d*np.cos(phi_i), 2*d*np.sin(phi_i)) for d in dd for phi_i in np.arange(0, 2*np.pi, np.pi/(3*d))]

In [125]:
plt.close('all')

fig, ax = plt.subplots(figsize=(8,8))
ax.set_xlim(left=-15, right=15)
ax.set_ylim(bottom=-15, top=15)
for x, y in centri:
    hex = RegularPolygon((x, y), numVertices=6, radius=1.1, alpha=0.2,
                         orientation=np.radians(60), edgecolor='k')
    ax.add_patch(hex)
    # Also add a text label
#     ax.text(x, y+0.2, l[0], ha='center', va='center', size=20)

# Also add scatter points in hexagon centres
# ax.scatter(hcoord, vcoord, c=[c[0].lower() for c in colors], alpha=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
import numpy as np
import seaborn as sns
sns.set_theme(style="ticks")

rs = np.random.RandomState(11)
x = rs.gamma(2, size=10000)
y = -.5 * x + rs.normal(size=10000)

sns.jointplot(x=x, y=y, kind="hex", color="#4CB391")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
list(zip(fff.block_names, fff.b_off, fff.block_sizes))

[('WDF1', 0, 512),
 ('DATA', 512, 5521616),
 ('YLST', 5522128, 28),
 ('XLST', 5522156, 4084),
 ('WXDA', 5526240, 2605),
 ('ORGN', 5528845, 65444),
 ('TEXT', 5594289, 106),
 ('SURF', 5594395, 46192514),
 ('WXDM', 51786909, 498448),
 ('WHTL', 52285357, 267676),
 ('WMAP', 52553033, 64),
 ('BKXL', 52553097, 4100),
 ('WXCS', 52557197, 10897),
 ('WXIS', 52568094, 3933),
 ('ZLDC', 52572027, 314),
 ('WMSK', 52572341, 290)]

In [2]:
vis.AllMaps(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
vis.ShowSpectra(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
xlabel, ylabel = fff.map_params["StepSizes"][fff.map_params["StepSizes"] > 0]

array([1.e+03, 5.e-01], dtype=float32)

In [2]:
vis.ShowSelected(fff);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
nx=10
ny=5
arr = np.arange(nx*ny)#.reshape(ny,nx)
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [11]:
reordered = [arr[(xx // nx + 1)*nx - (xx % nx) -1] if (xx//nx) % 2 == 1 else arr[xx] for xx in range(nx*ny)]

In [12]:
np.array(reordered).reshape(ny,nx)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [39, 38, 37, 36, 35, 34, 33, 32, 31, 30],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [17]:
reorder_inverted(arr, nx, ny)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [39, 38, 37, 36, 35, 34, 33, 32, 31, 30],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [22]:
def reorder_inverted(arr, nx, ny):
    reordered = [arr[(xx // nx + 1)*nx - (xx % nx) -1] if (xx//nx) % 2 == 1 else arr[xx] for xx in range(nx*ny)]
    return np.array(reordered).reshape(ny, nx)
    
def reorder_inv(arr, nx, ny):
    arr = arr.reshape(ny, nx)
    return np.array([arr[i][::-1] if (i & 1) else arr[i] for i in range(ny)])

# or, if already in the right shape:

def reorder(arr):
    ny, nx = arr.shape
    return np.array([arr[i][::-1] if i&1 else arr[i] for i in range(ny)])

In [27]:
%timeit reorder_inverted(arr, nx, ny)

125 ms ± 8.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit reorder_inv(arr, nx, ny)

548 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
prd.reshape().ndim

1

In [31]:
prd = reorder_inv(arr, nx, ny)

In [32]:
reorder_inv(prd, nx, ny)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [29]:
np.all(reorder_inverted(arr, nx, ny) == reorder_inv(arr, nx, ny))

True

In [2]:
spectra = pp.correct_saturated(rawspectra, (ny, nx),
                               n_nearest_features=10, max_iter=64,
                               smoothen=True, lam=5, copy=True)

In [109]:
spectra[1853,444:452] = 50
spectra[2914, 880:885] = 55
spectra[1936, 193:199] = 65
spectra[2241, 862:866] = 65
spectra[2231, 177:182] = 65
spectra[2657, 203:208] = 65
spectra[1637, 290:295] = 80
spectra[918, 797:802] = 65
spectra[1510, 341:353] = 68

In [3]:
baseline = cc.baseline_als(spectra[:,120:], lam=80000)
# baseline -= np.min(baseline, axis=-1, keepdims=True)

spectra = spectra[:, 120:]

x_values = x_values[108:]

100%|██████████| 19881/19881 [01:26<00:00, 230.77it/s]


In [49]:
bl = vis.AllMaps(np.log(1+baseline.reshape(ny, nx, -1)), x_values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
vis.ShowSpectra(spectra-baseline, sigma=x_values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cosmic Rays search 

In [52]:
# spectra -= cc.baseline_als(spectra, lam=200)
spectra -= np.min(spectra, axis=-1, keepdims=True)
spectra2 = (spectra - baseline) / \
            np.median(spectra - baseline, axis=-1, keepdims=True)
sisa = (np.max(spectra2, axis=0)-2*np.percentile(spectra2, 99, axis=0)
        +np.median(spectra2, axis=0))/np.median(spectra2, axis=0)
st_dev = np.std(sisa)

suspicious_positions = np.where(sisa > 2*st_dev)[0]
condition = spectra2.T[suspicious_positions] > 10*np.median(spectra2, axis=-1, keepdims=True)[suspicious_positions]
uslov = np.nonzero(condition)
suspects = np.unique([uslov[1][x] for x, y in itertools.groupby(uslov[0]) if len(list(y))<=np.ceil(len(uslov[0])/100)])
zez = vis.ShowSpectra(spectra2[suspects]);

IndexError: arrays used as indices must be of integer (or boolean) type

In [53]:
uslov

(array([], dtype=int64), array([], dtype=int64))

In [54]:
np.where(sisa > 2*st_dev)

(array([152, 153, 328, 416, 417, 418, 419, 420, 422]),)

In [31]:
len(suspicious_positions), condition.shape, spectra.shape, sisa.shape

(27, (27, 4347), (4347, 1015), (1015,))

In [34]:
np.where(sisa > 2*st_dev)

(array([ 33, 129, 169, 179, 180, 195, 196, 197, 198, 205, 206, 291, 292,
        293, 448, 582, 583, 799, 800, 863, 864, 865, 881, 882, 883, 884,
        922]),)

In [28]:
uslov = np.nonzero(condition)

In [32]:
uslov

(array([ 0,  1,  2,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  5,  6,  7,  8,  9,  9, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17,
        18, 19]),
 array([  43,  494,  494, 1310,   37,  159,  479,  519,  600,  640,  681,
         759,  870,  896, 1101, 1103, 1141, 1143,  965,  965,  809,   44,
          44,  912,   44,  912,   44,  912,  119,    4,  608,  608, 1262,
         495,  329, 1146]))

In [30]:
np.ce

array([  43.,  494.,  494., 1310.,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
        965.,  965.,  809.,   44.,   44.,  912.,   44.,  912.,   44.,
        912.,  119.,    4.,  608.,  608., 1262.,  495.,  329., 1146.])

In [27]:
np.delete(suspicious_positions, 4)

array([ 12,  70,  71, 136, 314, 315, 348, 365, 366, 367, 368, 383, 389,
       590, 591, 630, 742, 745, 784])

In [38]:
[uslov[1][x] for x, y in itertools.groupby(uslov[0]) if len(list(y))<=np.ceil(len(uslov[0])/100)]

[43,
 494,
 494,
 1310,
 159,
 479,
 519,
 600,
 870,
 896,
 1101,
 1103,
 1141,
 1143,
 965,
 965]

In [31]:
import itertools
np.where(uslov[0] != 4, uslov[1], np.nan)

In [55]:
vis.ShowSpectra(spectra2.T[suspicious_positions])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
vis.ShowSpectra(spectra.T[suspicious_positions])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Saturated & Zero spectra

In [4]:
plt.close("all")
trt = pp.find_saturated(rawspectra)
almostraw = pp.correct_zeros(rawspectra, copy=True)
sat_idx = np.where(almostraw==0)
almostraw[sat_idx] = np.nan
novica = np.zeros_like(almostraw)
novica[sat_idx] = spectra[sat_idx]
novica[novica==0] = np.nan
ss = np.stack((rawspectra[trt]-10000, spectra[trt], novica[trt]), axis=-1)
vis.ShowSpectra(ss, x_values, title=["Spectra N°"+str(x) for x in trt]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# =============================================================================
#                                     PCA...
# =============================================================================
npca = 50
pca = decomposition.PCA(n_components=npca)
pca_fit = pca.fit(spectra)

spectra_reduced = pca_fit.transform(spectra)
spectra_denoised = pca_fit.inverse_transform(spectra_reduced)

guz = pca.get_covariance()

In [4]:
vis.AllMaps(spectra_reduced.reshape(ny, nx, -1), components=pca.components_, components_sigma=x_values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()
plt.plot(guz[:,674])
# plt.plot(guz[:,179])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
plt.close('all')
plt.imshow(np.log(guz))
# plt.imshow(np.log(guz))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.close('all')
plt.figure()
plt.plot(spectra[674])
plt.plot(rawspectra[674]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.figure()
plt.plot(np.log(pca.explained_variance_ratio_), 'bo')
plt.plot(np.log(pca.explained_variance_), 'xg')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# =============================================================================
#                  showing the smoothed spectra
# =============================================================================
valid_idx = np.ones(npca, dtype=bool)
valid_idx[1] = 0
novica = pca.mean_ + np.dot(spectra_reduced[:, valid_idx], pca.components_[valid_idx])

In [ ]:
vis.AllMaps(novica.reshape(ny, nx, -1), sigma=x_values)

In [ ]:
vis.ShowSpectra(novica, sigma=x_values)

```Python
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from time import time

estimators = [
    BayesianRidge(),
    DecisionTreeRegressor(max_features='sqrt'),
    ExtraTreesRegressor(n_estimators=10),
    KNeighborsRegressor(n_neighbors=8)
]


zero_idx = pp.find_zeros(rawspectra)
rawspectra[zero_idx] = np.median(rawspectra, axis=0)

sat = pp.find_saturated(rawspectra)

saturated_idx = np.where(rawspectra==0)

# rawspectra[saturated_idx] = np.nan
rawspectra[saturated_idx] = np.nan

trt = pp.get_neighbourhood(sat, map_shape)
# create an array so that trt[vrackalica] = sat
vrackalica = np.array([np.argwhere(trt==i)[0][0] for i in sat])

krk = dict()
tajming = dict()
for jj, est in enumerate(estimators):
    imp = impute.IterativeImputer(n_nearest_features=8, max_iter=44, estimator=est)
    start = time()
    popravljeni = imp.fit_transform(rawspectra[trt].T).T[vrackalica]
    tajming[str(est)] = time()-start
    spectra = np.copy(rawspectra)
    spectra[sat] = popravljeni
    ss = np.stack((rawspectra[sat]-1000, spectra[sat]), axis=-1)
    krk[jj] = vis.ShowSpectra(ss, x_values, title=[str(x) for x in sat])
    krk[jj].fig.suptitle(str(est))
```

In [ ]:
# if smooth:
#     upeglani = cc.baseline_als(popravljeni, lam=len(x_values)//5, p=0.6)

#     is_changed = np.diff(saturated_idx[0], prepend=sat[0])!=0
#     renormalizovani = []
#     i = 0
#     for cond in is_changed:
#         if cond:
#             i+=1
#         renormalizovani.append(i)

#     novi = np.copy(saturated_idx)
#     novi[0] = np.array(renormalizovani)
#     novi = tuple(novi)


#     spectra[saturated_idx] = upeglani[novi]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
np.meshgrid(np.arange(ny), np.arange(nx), indexing='ij')

[array([[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 2,  2,  2, ...,  2,  2,  2],
        ...,
        [60, 60, 60, ..., 60, 60, 60],
        [61, 61, 61, ..., 61, 61, 61],
        [62, 62, 62, ..., 62, 62, 62]]),
 array([[ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        ...,
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68],
        [ 0,  1,  2, ..., 66, 67, 68]])]

In [139]:
plt.figure()
plt.plot(spectra[595])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [231]:
ll = []
for i in ll:
    print("evo")